In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.read_pickle('../input/feats/df_trn_feat0.pkl')
df_test  = pd.read_pickle('../input/feats/df_test_feat0.pkl')

how to do stratified kfold in continuous variable

mean encode

- by categorical 

In [3]:
import gc
from sklearn.model_selection import StratifiedKFold, KFold

In [ ]:
def mean_expand_train(df,cols,target='deal_probability'):
    for col in cols:
        cumsum = df.groupby(col)[target].cumsum() - df[col]
        cumcnt = df

In [62]:
def mean_LOO_train(df,cols,target='deal_probability',seed=0):
    df_new = df.copy()
    print('Leave one out mean encode...')
    for col in cols:
        print(col)
        grouped = df_new.groupby(col)
        n = grouped[target].transform('count')
        mean = grouped[target].transform('mean')
        
        df_new[col+'_mean_target'] = (mean*n - df_new[target])/(n-1)
    
    prior = df[target].mean()
    mean_target_cols = [col+'_mean_target' for col in cols]
    df_new[mean_target_cols] = df_new[mean_target_cols].fillna(prior)
    print('done')
    return df_new[mean_target_cols]

In [12]:
kf = KFold(shuffle=True,random_state=0)
kf.split(df_train)
# for e in kf.split(df_train):
#     print(e)

<generator object _BaseKFold.split at 0x00000081D664D9E8>

In [13]:
def mean_encode_train(df, cols, nfolds=4, num_bucket=100, target='deal_probability',seed=0):
    df_new = df.copy()
    
    
    ## create empty col
    for col in cols:
        df_new[col+'_mean_target'] = 0
    
    nfold_indx = {}
    y_tr = df_new[target].values
    
    ## stratified k fold in continus y
    if num_bucket:
        percentiles = np.percentile(y_tr, np.linspace(0, 100, num_bucket)) ## create percentiles 
        falls_into = np.searchsorted(percentiles,y_tr) ## fall into bucket

        skf = StratifiedKFold(nfolds, random_state=seed)
        skf.get_n_splits(df_new, falls_into)
        skf_gen = skf.split(df_new, falls_into)
    ## K fold     
    else:
        skf = KFold(nfolds,shuffle=True,random_state=seed)
        skf_gen = skf.split(df_new)
        
    for fold, (tr_idx, val_idx) in enumerate(skf_gen):
        nfold_indx[fold] = {'tr_idx':tr_idx,'val_idx':val_idx}
        
        X_tr ,X_val = df_new.iloc[tr_idx].copy(), df_new.iloc[val_idx].copy()
        
        
        print('fold:{} mean encoding...'.format(fold),end='\n\t')
        
        for col in cols:
            print(col,end='\t')
            means = X_val[col].map(X_tr.groupby(col)[target].mean()) ## map mean encoding in X_tr to X_val
            X_val[col + '_mean_target'] = means.astype('float16')

        df_new.iloc[val_idx] = X_val
        print('')    
        del X_tr,X_val;gc.collect()

    prior = df[target].mean()
    mean_target_cols = [col+'_mean_target' for col in cols]
    df_new[mean_target_cols] = df_new[mean_target_cols].fillna(prior)
    
    print('complete')
    return df_new[mean_target_cols],nfold_indx

In [6]:
def mean_encode_test(df_train, df_test, cols,target='deal_probability'):
    ''' encode df_test mean target by df_train'''
    y_target = df_train[target]
    
    for col in cols:
        print(col,end='\t')
        means = df_test[col].map(df_train.groupby(col)[target].mean()) ## map mean encode in train to test
        df_test[col+'_mean_target'] = means.astype('float16')
    print('')
    prior = df_train[target].mean()
    mean_target_cols = [col+'_mean_target' for col in cols]
    
    df_test[mean_target_cols] = df_test[mean_target_cols].fillna(prior)
    print('complete')
    return df_test[mean_target_cols]

In [14]:
df_train.columns

Index(['uidx', 'iidx', 'iid', 'region_city_label', 'tit_len', 'desc_len',
       'activation_date', 'month', 'day', 'weekday', 'param_1', 'param_2',
       'param_3', 'user_type', 'parent_category_name', 'price',
       'category_name', 'image_top_1', 'ads_cnt_by_uid', 'ads_cnt_by_iid',
       'deal_probability'],
      dtype='object')

In [16]:
encode_cols = [
    'uidx','iidx',
    'region_city_label',    
    'param_1',
#     'param_2','param_3',
    'image_top_1',## added
#     'month','day','weekday',
    'user_type',
    'parent_category_name','category_name'
]
df_train_mean_target, nfold_idx = mean_encode_train(df_train,cols=encode_cols,nfolds=4,num_bucket=None) ## 
# df_train_mean_LOO = mean_LOO_train(df_train,cols=encode_cols) ##

fold:0 mean encoding...
	uidx	iidx	region_city_label	param_1	image_top_1	user_type	parent_category_name	category_name	
fold:1 mean encoding...
	uidx	iidx	region_city_label	param_1	image_top_1	user_type	parent_category_name	category_name	
fold:2 mean encoding...
	uidx	iidx	region_city_label	param_1	image_top_1	user_type	parent_category_name	category_name	
fold:3 mean encoding...
	uidx	iidx	region_city_label	param_1	image_top_1	user_type	parent_category_name	category_name	
complete


In [17]:
df_nfold_idx = pd.DataFrame(nfold_idx)
df_nfold_idx

,0,1,2,3
tr_idx,"[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 14, 15, 16...","[0, 3, 4, 7, 8, 9, 11, 12, 13, 15, 16, 17, 19,...","[1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 16, ...","[0, 1, 2, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 1..."
val_idx,"[7, 12, 13, 19, 20, 24, 31, 36, 42, 43, 47, 53...","[1, 2, 5, 6, 10, 14, 18, 22, 27, 30, 37, 45, 4...","[0, 8, 9, 15, 17, 23, 25, 29, 32, 34, 41, 48, ...","[3, 4, 11, 16, 21, 26, 28, 33, 35, 38, 39, 40,..."


In [18]:
df_test_mean_target = mean_encode_test(df_train, df_test,cols=encode_cols)

uidx	iidx	region_city_label	param_1	image_top_1	user_type	parent_category_name	category_name	
complete


In [19]:
print('shape of df_train_mean_target:',df_train_mean_target.shape)
print('shape of df_test_mean_target:',df_test_mean_target.shape)

shape of df_train_mean_target: (1503424, 8)
shape of df_test_mean_target: (508438, 8)


In [20]:
store = pd.HDFStore('../input/feats/df_mean_target.h5') 
store['df_trn_mean_target'] = df_train_mean_target
store['df_test_mean_target'] = df_test_mean_target
store['df_nfold_idx'] = df_nfold_idx
# store['df_trn_mean_LOO'] = df_train_mean_LOO
store.close()

C:\Program Files\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2862: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->[0, 1, 2, 3]]

  exec(code_obj, self.user_global_ns, self.user_ns)
